#### Importación de paquetes

In [18]:
import cv2
import FaceDetectors

#### Comparación de gafas con detector facial basado en Convolutional Neural Networks (CNNs)

In [20]:
# Inicializar el detector facial
FDet = FaceDetectors.FaceDetector()

# Cargar las imágenes de las gafas
gafas1 = cv2.imread('Images/aviador.png', cv2.IMREAD_UNCHANGED)
gafas2 = cv2.imread('Images/deportivas.png', cv2.IMREAD_UNCHANGED)
gafas3 = cv2.imread('Images/estilizadas.png', cv2.IMREAD_UNCHANGED)
gafas4 = cv2.imread('Images/fashion.png', cv2.IMREAD_UNCHANGED)
gafas5 = cv2.imread('Images/modernas.png', cv2.IMREAD_UNCHANGED)

# Redimensionamos las imágenes de las gafas
gafas1 = cv2.resize(gafas1, (gafas1.shape[1] // 5, gafas1.shape[0] // 5))
gafas2 = cv2.resize(gafas2, (gafas2.shape[1] // 5, gafas2.shape[0] // 5))
gafas3 = cv2.resize(gafas3, (gafas3.shape[1] // 7, gafas3.shape[0] // 7))
gafas4 = cv2.resize(gafas4, (gafas4.shape[1] // 4, gafas4.shape[0] // 4))
gafas5 = cv2.resize(gafas5, (gafas5.shape[1] // 5, gafas5.shape[0] // 5))

# Lista con las gafas para intercambiar entre ellas
gafas = [gafas1, gafas2, gafas3, gafas4, gafas5]

# Variable que controla que gafa se visualiza
control_gafas = 0

# Conectar a la cámara web
cap = cv2.VideoCapture(0)

# Verificar si hay otras cámaras disponibles
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Error de cam')
            exit(0)
        else:
            print('Cam 0')
    else:
        print('Cam 1')
else:
    print('Cam 0')

# Establecer la resolución de la cámara
cap.set(3, 640)
cap.set(4, 480)

def superponer_filtro(frame, filtro, shape_index, x_offset, y_offset):
    x_filtro = shape[shape_index][0] - int(filtro.shape[1] / 2)
    y_filtro = shape[shape_index][1] - int(filtro.shape[0] / 2) + y_offset

    if x_filtro >= 0 and y_filtro >= 0 and \
    (x_filtro + filtro.shape[1] <= frame.shape[1]) and \
    (y_filtro + filtro.shape[0] <= frame.shape[0]):
        
        filtro_resized = cv2.resize(filtro, (int(filtro.shape[1]), int(filtro.shape[0])))
        alpha = filtro_resized[:, :, 3] / 255.0

        for c in range(0, 3):
            frame[y_filtro:y_filtro + filtro_resized.shape[0], x_filtro:x_filtro + filtro_resized.shape[1], c] = \
            (
                frame[y_filtro:y_filtro + filtro_resized.shape[0], x_filtro:x_filtro + filtro_resized.shape[1], c] * 
                (1 - alpha) + filtro_resized[:, :, c] * alpha
            )

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    # Detección de rostro y ojos en el fotograma
    values = FDet.DetectLargestFaceEyesDNN(frame)
    if values is not None:
        face, eyes, shape = values

        # Dibujar un marco alrededor del rostro
        [x, y, w, h] = face
        if x > -1:
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                superponer_filtro(frame, gafas[control_gafas], 27, 0, 10)

    cv2.imshow('Cam', frame)
    
    # Esc para finalizar
    tec = cv2.waitKey(40)

    if tec == 27:  # Esc
        break

    # Cambio de gafas
    elif tec & 0xFF == ord('d'):
        control_gafas += 1
        if control_gafas >= len(gafas):
            control_gafas = 0

# Cerrar la ventana y liberar la cámara
cap.release()
cv2.destroyAllWindows()


Cam 0
